<a href="https://colab.research.google.com/github/abhiram172/umbc_data690_fall2024_1/blob/main/Assignment/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10/30/2024

In [1]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", None)

## Define reusable functions

In [2]:
FIRST = "First Quartile (<=25%)"
SECOND = "Second Quartile (25-50%)"
THIRD = "Third Quartile (50-75%)"
FOURTH = "Fourth Quartile (>75%)"

def assign_quartile(value, x, y, z):
    if value <= x:
        quartile = FIRST
    elif value > x and value <= y:
        quartile = SECOND
    elif value > y and value <= z:
        quartile = THIRD
    else:
        quartile = FOURTH
    return quartile

## Load and Explore Data

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/wcj365/python-stats-dataviz/refs/heads/master/fall2024/data/World_Development_Indicators_(WDI).csv")

print(df.shape)
df.sample(3)

(4123, 9)


,Year,Country,GDP per capita (current US$),"Life expectancy at birth, total (years)","Population, total",Country Code,Region,Income Group,Lending Type
2758,2007,North Macedonia,4079.390404,73.800,2043559.0,MKD,Europe & Central Asia,Upper middle income,IBRD
3749,2010,Tunisia,4241.011910,75.421,10895063.0,TUN,Middle East & North Africa,Lower middle income,IBRD
2544,2021,Morocco,3767.524902,74.042,37076584.0,MAR,Middle East & North Africa,Lower middle income,IBRD


In [4]:
column_names_dict = {
    "GDP per capita (current US$)" : "GDP per Capita",
    "Life expectancy at birth, total (years)" : "Life Expectancy",
    "Population, total" : "Population"
}

df = df.rename(columns=column_names_dict)
df.sample(3)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
4033,2009,Virgin Islands (U.S.),38753.182539,77.514634,108404.0,VIR,Latin America & Caribbean,High income,Not classified
2872,2007,Panama,6205.821576,76.154000,3431614.0,PAN,Latin America & Caribbean,High income,IBRD
1493,2015,Grenada,8379.621847,75.013000,118980.0,GRD,Latin America & Caribbean,Upper middle income,Blend


In [5]:
df.sample(3).T

,1785,4102,867
Year,2022,2021,2016
Country,Ireland,Zambia,"Congo, Rep."
GDP per Capita,103983.291336,1134.713454,2107.503024
Life Expectancy,NaN,61.223,63.577
Population,5127170.0,19473125.0,5186824.0
Country Code,IRL,ZMB,COG
Region,Europe & Central Asia,Sub-Saharan Africa,Sub-Saharan Africa
Income Group,High income,Lower middle income,Lower middle income
Lending Type,Not classified,IDA,Blend


## Homework Question 3 (bonus, will cover next week)

step 1 - create a column "Life Expectancy Quartile" similar to the GDP per Capita Quartile above.
Step 2 - Create a column "health and wealth status" based on the following definition
- rich and healthy (1st GDP per capita quartile and 1st life expentancy quartile)
- poor and unhealthy ((4th GDP per capita quartile and 4th life expentancy quartile)
- rich and unhealthy (1st GDP per capita quartile and 4th life expentancy quartile)
- poor and healthy ((4th GDP per capita quartile and 1st life expentancy quartile)
- Other

In [27]:
import pandas as pd

# Load the data
df = pd.read_csv("https://raw.githubusercontent.com/wcj365/python-stats-dataviz/refs/heads/master/fall2024/data/World_Development_Indicators_(WDI).csv")

# Renaming columns for easier reference
column_names_dict = {
    "GDP per capita (current US$)": "GDP per Capita",
    "Life expectancy at birth, total (years)": "Life Expectancy",
    "Population, total": "Population"
}
df = df.rename(columns=column_names_dict)

# Filter the data for the year 2020 and drop rows with missing GDP or Life Expectancy values
df_2020 = df[df["Year"] == 2020].dropna(subset=["GDP per Capita", "Life Expectancy"])

# Define quartile thresholds for Life Expectancy and GDP per Capita
life_q1, life_q2, life_q3 = df_2020['Life Expectancy'].quantile([0.25, 0.5, 0.75])
gdp_q1, gdp_q2, gdp_q3 = df_2020['GDP per Capita'].quantile([0.25, 0.5, 0.75])

# Function to assign quartiles
def assign_quartile(value, q1, q2, q3):
    if value <= q1:
        return "First Quartile"
    elif value <= q2:
        return "Second Quartile"
    elif value <= q3:
        return "Third Quartile"
    else:
        return "Fourth Quartile"

# Assign Life Expectancy Quartile and GDP Quartile
df_2020['Life Expectancy Quartile'] = df_2020['Life Expectancy'].apply(assign_quartile, args=(life_q1, life_q2, life_q3))
df_2020['GDP Quartile'] = df_2020['GDP per Capita'].apply(assign_quartile, args=(gdp_q1, gdp_q2, gdp_q3))


In [28]:
# Function to determine Health and Wealth Status based on specified criteria
def health_wealth_status(row):
    if row['GDP Quartile'] == "First Quartile" and row['Life Expectancy Quartile'] == "First Quartile":
        return "Rich and healthy"
    elif row['GDP Quartile'] == "Fourth Quartile" and row['Life Expectancy Quartile'] == "Fourth Quartile":
        return "Poor and unhealthy"
    elif row['GDP Quartile'] == "First Quartile" and row['Life Expectancy Quartile'] == "Fourth Quartile":
        return "Rich and unhealthy"
    elif row['GDP Quartile'] == "Fourth Quartile" and row['Life Expectancy Quartile'] == "First Quartile":
        return "Poor and healthy"
    else:
        return "Other"

# Apply the health_wealth_status function
df_2020['Health and Wealth Status'] = df_2020.apply(health_wealth_status, axis=1)

# Display the results to verify
df_2020[['Country', 'GDP per Capita', 'Life Expectancy', 'GDP Quartile', 'Life Expectancy Quartile', 'Health and Wealth Status']].head(10)


,Country,GDP per Capita,Life Expectancy,GDP Quartile,Life Expectancy Quartile,Health and Wealth Status
16,Afghanistan,512.055098,62.575000,First Quartile,First Quartile,Rich and healthy
35,Albania,5343.037704,76.989000,Second Quartile,Third Quartile,Other
54,Algeria,3354.153164,74.453000,Second Quartile,Third Quartile,Other
111,Angola,1450.905112,62.261000,First Quartile,First Quartile,Rich and healthy
130,Antigua and Barbuda,15224.858589,78.841000,Third Quartile,Fourth Quartile,Other
149,Argentina,8500.837939,75.892000,Third Quartile,Third Quartile,Other
168,Armenia,4505.867742,72.173000,Second Quartile,Second Quartile,Other
187,Aruba,24008.127822,75.723000,Fourth Quartile,Third Quartile,Other
206,Australia,51868.247557,83.200000,Fourth Quartile,Fourth Quartile,Poor and unhealthy
225,Austria,48789.497850,81.192683,Fourth Quartile,Fourth Quartile,Poor and unhealthy


In [29]:
import plotly.express as px

# Create the choropleth map
fig = px.choropleth(
    df_2020,
    locations="Country",
    locationmode="country names",
    color="Health and Wealth Status",
    hover_name="Country",
    title="Health and Wealth Status of Countries (2020)",
    color_discrete_map={
        "Rich and healthy": "green",
        "Poor and unhealthy": "red",
        "Rich and unhealthy": "orange",
        "Poor and healthy": "blue",
        "Other": "gray"
    }
)

# Show the plot
fig.show()


In [6]:
print(df.shape)
df.sample(5)

(4123, 9)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
952,2006,Curacao,NaN,74.709756,141239.0,CUW,Latin America & Caribbean,High income,Not classified
3821,2006,Tuvalu,2402.480106,63.928000,10030.0,TUV,East Asia & Pacific,Upper middle income,IDA
2511,2007,Montenegro,5976.394145,74.339024,615875.0,MNE,Europe & Central Asia,Upper middle income,IBRD
3389,2011,South Sudan,1455.358407,54.940000,10243050.0,SSD,Sub-Saharan Africa,Low income,IDA
1506,2009,Guam,29335.277677,77.094000,164580.0,GUM,East Asia & Pacific,High income,Not classified


In [7]:
df_2020 = df[df["Year"] == 2020]
print(df_2020.shape)
df_2020.sample(5)

(217, 9)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
871,2020,"Congo, Rep.",2011.269479,63.785,5702174.0,COG,Sub-Saharan Africa,Lower middle income,Blend
2752,2020,Nigeria,2074.613747,52.887,208327405.0,NGA,Sub-Saharan Africa,Lower middle income,Blend
548,2020,Brunei Darussalam,27179.352887,74.795,441725.0,BRN,East Asia & Pacific,High income,Not classified
4101,2020,Zambia,956.831729,62.380,18927715.0,ZMB,Sub-Saharan Africa,Lower middle income,IDA
1593,2020,Guyana,6863.074346,68.486,797202.0,GUY,Latin America & Caribbean,High income,IDA


In [8]:
df_2020 = df_2020.dropna(subset=["GDP per Capita", "Life Expectancy"])
df_2020.shape

(202, 9)

In [9]:
# Step 1: Creates 'Life Expectancy Quartile' column and lists the quartile the row falls under.
df_2020['Life Expectancy'].describe()



,Life Expectancy
count,202.000000
mean,72.314599
std,7.451962
min,52.777000
25%,66.779750
50%,72.871500
75%,77.981750
max,85.497561


In [10]:
x = df_2020['Life Expectancy'].describe()['25%']
y = df_2020['Life Expectancy'].describe()['50%']
z = df_2020['Life Expectancy'].describe()['75%']

In [11]:
df_2020['Life Expectancy Quartile'] = df_2020['Life Expectancy'].apply(assign_quartile, args=(x, y, z,))
df_2020.sample(5)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile
3360,2020,Somalia,556.578066,55.967000,16537016.0,SOM,Sub-Saharan Africa,Low income,IDA,First Quartile (<=25%)
225,2020,Austria,48789.497850,81.192683,8916864.0,AUT,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%)
2942,2020,Peru,6063.626923,73.665000,33304756.0,PER,Latin America & Caribbean,Upper middle income,IBRD,Third Quartile (50-75%)
3455,2020,St. Kitts and Nevis,18553.657137,71.627000,47642.0,KNA,Latin America & Caribbean,High income,IBRD,Second Quartile (25-50%)
3550,2020,Suriname,4796.533314,72.561000,607065.0,SUR,Latin America & Caribbean,Upper middle income,IBRD,Second Quartile (25-50%)


In [12]:
x = df_2020['GDP per Capita'].describe()['25%']
y = df_2020['GDP per Capita'].describe()['50%']
z = df_2020['GDP per Capita'].describe()['75%']

print(x, y, z)

2188.047693164725 5920.260418960404 20761.21119943435


In [13]:
# Step 2: Creates 'GDP Quartile' column and lists the quartile the row falls under.
# Next, creates 'Health and Wealth Status' column which categorizes a row if
# a country and a year falls under (4) different categories.


df_2020['GDP Quartile'] =  df_2020['GDP per Capita'].apply(assign_quartile, args=(x, y, z,))
df_2020.sample(10)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile
567,2020,Bulgaria,10148.342395,73.658537,6934015.0,BGR,Europe & Central Asia,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%)
2619,2020,Nauru,10124.700622,63.437000,12315.0,NRU,East Asia & Pacific,High income,IBRD,First Quartile (<=25%),Third Quartile (50-75%)
1935,2020,Kenya,1936.250755,62.675000,51985780.0,KEN,Sub-Saharan Africa,Lower middle income,Blend,First Quartile (<=25%),First Quartile (<=25%)
301,2020,Bangladesh,2233.305901,71.968000,167420951.0,BGD,South Asia,Lower middle income,IDA,Second Quartile (25-50%),Second Quartile (25-50%)
111,2020,Angola,1450.905112,62.261000,33428486.0,AGO,Sub-Saharan Africa,Lower middle income,IBRD,First Quartile (<=25%),First Quartile (<=25%)
719,2020,Central African Republic,435.469248,54.604000,5343020.0,CAF,Sub-Saharan Africa,Low income,IDA,First Quartile (<=25%),First Quartile (<=25%)
1308,2020,France,39179.744260,82.175610,67571107.0,FRA,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%)
2391,2020,Mauritania,1836.292411,64.532000,4498604.0,MRT,Sub-Saharan Africa,Lower middle income,IDA,First Quartile (<=25%),First Quartile (<=25%)
909,2020,Cote d'Ivoire,2349.069882,59.032000,26811790.0,CIV,Sub-Saharan Africa,Lower middle income,IDA,First Quartile (<=25%),Second Quartile (25-50%)
1080,2020,Dominican Republic,7167.914974,72.889000,10999664.0,DOM,Latin America & Caribbean,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%)


In [14]:
def find_status(row):
  if (row['Life Expectancy Quartile'] == FIRST) and (row['GDP Quartile'] == FIRST):
    return 'Poor and unhealthy'
  elif (row['Life Expectancy Quartile'] == FOURTH) and (row['GDP Quartile'] == FIRST):
    return 'Poor but healthy'
  elif (row['Life Expectancy Quartile'] == FOURTH) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and healthy'
  elif (row['Life Expectancy Quartile'] == THIRD) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and somewhat healthy'
  elif (row['Life Expectancy Quartile'] == SECOND) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and somewhat unhealthy'
  elif (row['Life Expectancy Quartile'] == FIRST) and (row['GDP Quartile'] == FOURTH):
    return 'Rich but unhealthy'
  else:
    return 'Other'

df_2020['Health and Wealth Status'] = df_2020.apply(find_status,axis=1)
df_2020.sample(10)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
2980,2020,Poland,15816.820402,76.500000,37899070.0,POL,Europe & Central Asia,High income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%),Other
358,2020,Belgium,45609.003494,80.695122,11538604.0,BEL,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
1726,2020,Indonesia,3895.618152,68.808000,271857970.0,IDN,East Asia & Pacific,Upper middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%),Other
1650,2020,"Hong Kong SAR, China",46109.229995,85.497561,7481000.0,HKG,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
1612,2020,Haiti,1283.141228,64.052000,11306801.0,HTI,Latin America & Caribbean,Lower middle income,IDA,First Quartile (<=25%),First Quartile (<=25%),Poor and unhealthy
1118,2020,"Egypt, Arab Rep.",3571.556907,70.990000,107465134.0,EGY,Middle East & North Africa,Lower middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%),Other
149,2020,Argentina,8500.837939,75.892000,45376763.0,ARG,Latin America & Caribbean,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%),Other
567,2020,Bulgaria,10148.342395,73.658537,6934015.0,BGR,Europe & Central Asia,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%),Other
1403,2020,Germany,46749.476228,81.041463,83160871.0,DEU,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2296,2020,Malaysia,10164.344431,75.938000,33199993.0,MYS,East Asia & Pacific,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%),Other


In [15]:
df_2020["Health and Wealth Status"].value_counts()

,count
Health and Wealth Status,
Other,113
Rich and healthy,42
Poor and unhealthy,38
Rich and somewhat healthy,7
Rich and somewhat unhealthy,2


# Countries that belong to each status category.

In [16]:
df_rich_healthy = df_2020[df_2020["Health and Wealth Status"] == "Rich and healthy"]
print(df_rich_healthy.shape)
df_rich_healthy.sample(5)

(42, 12)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
1004,2020,Czechia,22992.879383,78.226829,10697858.0,CZE,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
3569,2020,Sweden,52837.903978,82.356098,10353442.0,SWE,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2676,2020,New Caledonia,34877.635635,80.834951,271080.0,NCL,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
3018,2020,Puerto Rico,31427.429114,78.041000,3281557.0,PRI,Latin America & Caribbean,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
4044,2020,Virgin Islands (U.S.),39411.045254,79.819512,106290.0,VIR,Latin America & Caribbean,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy


In [17]:
df_rich_healthy["Country"].unique()

array(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Bermuda', 'Canada',
       'Channel Islands', 'Cyprus', 'Czechia', 'Denmark', 'Estonia',
       'Faroe Islands', 'Finland', 'France', 'Germany',
       'Hong Kong SAR, China', 'Iceland', 'Ireland', 'Isle of Man',
       'Israel', 'Italy', 'Japan', 'Korea, Rep.', 'Liechtenstein',
       'Luxembourg', 'Macao SAR, China', 'Malta', 'Netherlands',
       'New Caledonia', 'New Zealand', 'Norway', 'Portugal',
       'Puerto Rico', 'Qatar', 'Singapore', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'United Arab Emirates', 'United Kingdom',
       'Virgin Islands (U.S.)'], dtype=object)

In [18]:
df_poor_unhealthy = df_2020[df_2020["Health and Wealth Status"] == "Poor and unhealthy"]
print(df_poor_unhealthy.shape)
df_poor_unhealthy["Country"].unique()

(38, 12)


array(['Afghanistan', 'Angola', 'Benin', 'Burkina Faso', 'Burundi',
       'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Ethiopia', 'Gambia, The',
       'Ghana', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Kenya', 'Lesotho',
       'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritania',
       'Mozambique', 'Niger', 'Nigeria', 'Pakistan', 'Rwanda',
       'Sierra Leone', 'Somalia', 'Sudan', 'Tanzania', 'Togo', 'Uganda',
       'Yemen, Rep.', 'Zambia', 'Zimbabwe'], dtype=object)

In [19]:
df_poor_healthy = df_2020[df_2020["Health and Wealth Status"] == "Poor but healthy"]
print(df_poor_healthy.shape)
df_poor_healthy["Country"].unique()

(0, 12)


array([], dtype=object)

In [20]:
df_rich_unhealthy = df_2020[df_2020["Health and Wealth Status"] == "Rich but unhealthy"]
print(df_rich_unhealthy.shape)
df_rich_unhealthy["Country"].unique()

(0, 12)


array([], dtype=object)

In [21]:
df_rich_unhealthy

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status


In [22]:
df_other = df_2020[df_2020["Health and Wealth Status"] == "Other"]
df_other["Country"].nunique()

113

# Choropleth - Chorom (colorful), pleth -map, place.

Plot population of countries over a map using plotly

In [23]:
# Create the plot
fig = px.choropleth(
    df_2020,
    locations="Country",
    locationmode='country names',
    color="Health and Wealth Status",
    hover_name="Country",
#    color_continuous_scale=px.colors.sequential.Inferno
)

# Show the plot
fig.show()